In [ ]:
import json
import math
from pathlib import Path
import pickle
import re

import matplotlib.pyplot as plt
import numpy
import pandas
from pandas import DataFrame
from scipy.optimize import curve_fit


# Physical dimensions of the mouse
setup_source = Path('../../src/setup.h').read_text()
MOUSE_TAIL = float(re.findall('MOUSE_TAIL (.*)\n', setup_source)[0])
MOUSE_HEAD = float(re.findall('MOUSE_HEAD (.*)\n', setup_source)[0])

# Where to start and stop the curve fit
CURVE_START = MOUSE_HEAD
CURVE_STOP = 0.27

# Where to start and stop the linear fit
LINEAR_START = MOUSE_HEAD + 0.01
LINEAR_STOP = 0.20


def log_as_dataframe(log):
    columns = ['timestamp', 'level', 'source', 'function', 'data']
    df = pandas.DataFrame(log, columns=columns)
    return df.set_index('timestamp').sort_index()


df = log_as_dataframe(pickle.load(open('../log.pkl', 'rb')))
df = df[df['function'] == 'log_front_sensors_calibration']
data = []
for value in df['data'].values:
    try:
        value = json.loads(value.replace('nan', 'NaN'))
        data.append(value)
    except json.JSONDecodeError:
        pass
df = DataFrame(data)
df['left_raw'] = df['left_raw_on'] - df['left_raw_off']
df['right_raw'] = df['right_raw_on'] - df['right_raw_off']
df['micrometers'] = df['micrometers'] - df['micrometers'][0]
df['micrometers'] = (180 * 2 - 12 - MOUSE_TAIL * 1000) * 1000 - df['micrometers']
df['meters'] = df['micrometers'] / 1000000
df = df[(df['meters'] > CURVE_START) & (df['meters'] < CURVE_STOP)]
df = df.set_index('meters')
df = df.sort_index(ascending=False)
df.head()

In [ ]:
# Matplotlib configuration
%matplotlib inline
plt.style.use('seaborn')

In [ ]:
df['reference_distance'] = df.index.values
df['left_distance_error'] = df['left_distance'] - df['reference_distance']
df['right_distance_error'] = df['right_distance'] - df['reference_distance']
df['zero_distance_error'] = 0

df[['left_distance_error', 'right_distance_error', 'zero_distance_error']].plot()
df[['left_raw', 'right_raw']].plot()
df[['left_raw_on', 'right_raw_on']].plot()
df[['left_raw_off', 'right_raw_off']].plot()

In [ ]:
def raw_to_distances(raw, a, b):
    return a / numpy.log(raw) - b


linear_df = df[(df.index > LINEAR_START) & (df.index < LINEAR_STOP)]

(left_a, left_b), _ = curve_fit(raw_to_distances,
                                linear_df['left_raw'].values,
                                linear_df.index.values)
(right_a, right_b), _ = curve_fit(raw_to_distances,
                                  linear_df['right_raw'].values,
                                  linear_df.index.values)

round_to = 3
print('#define SENSOR_FRONT_LEFT_A', round(left_a, round_to))
print('#define SENSOR_FRONT_LEFT_B', round(left_b, round_to))
print('#define SENSOR_FRONT_RIGHT_A', round(right_a, round_to))
print('#define SENSOR_FRONT_RIGHT_B', round(right_b, round_to))

In [ ]:
df['left_distance_fit'] = raw_to_distances(df['left_raw'].values, left_a, left_b)
df['right_distance_fit'] = raw_to_distances(df['right_raw'].values, right_a, right_b)
df['left_distance_fit_error'] = df['left_distance_fit'] - df['reference_distance']
df['right_distance_fit_error'] = df['right_distance_fit'] - df['reference_distance']

df[['left_distance_fit_error', 'right_distance_fit_error', 'zero_distance_error']].plot()